## Investigate anchors IoU
During training it was found that the accuracy of the model was highlt dependent on the
number of anchor boxes associated with each ground truth box.

When a single anchor box was associated with the ground truth box the model would simply
return no classes - only a few anchor boxes out of the 8,732 anchor boxes would be
associated with a class. Therefore, the network learnt the easy solution to the problem;
to simply assume all boxes are associated with no class.

To solve this I implemented an algorithm that associates all anchor boxes with an IoU
above a certain threshold with a ground truth box to be of that boxes class.

However, I believe that if too many anchor boxes are associated with the ground truth
box then this may have negative consequences on bounding box regression. That is, even
far away boxes have to be regressed to the ground truth box.

As a result, this notebook aims to investigate what IoU threshold should be used.

In [ ]:
# We want to
# 1. Get all labels
# 2. Get all anchors
# 3. Find the average number of anchors associated with each GT box at each IoU threshold
# 4. [Optional] investigate difficultly of transforming boxes to GT

In [ ]:
from pathlib import Path

import torch
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

from ssd.anchor_box_generator import AnchorBoxGenerator
from ssd.data import LetterboxTransform, SSDDataset
from ssd.utils import BoxUtils

In [ ]:
LABELS_DIR = Path("")
LABEL_FILES = list(LABELS_DIR.glob("*.txt"))

DEVICE = torch.device("cpu")
DTYPE = torch.float32

IMAGE_WIDTH = 300
IMAGE_HEIGHT = 300

SAMPLE_SIZE = min(10_000, len(LABEL_FILES))

### Analyse the matching of GT to anchor boxes

In [ ]:
transform = LetterboxTransform(IMAGE_WIDTH, IMAGE_HEIGHT, DTYPE)

anchor_generator = AnchorBoxGenerator(DEVICE, DTYPE)
anchors = anchor_generator.forward(
    1, [(38, 38), (19, 19), (10, 10), (5, 5), (3, 3), (1, 1)]
)
all_boxes: torch.Tensor = None  # type: ignore
num_objects = 0
missed_objects = 0
classed_anchors = 0
background_anchors = 0
empty_anchors = 0
num_anchors_per_gt: list[int] = []
for label_file in tqdm(LABEL_FILES[:SAMPLE_SIZE]):
    objects = SSDDataset.read_label_file(label_file, DEVICE, DTYPE)
    letterbox_objects = transform.transform_objects(objects, IMAGE_WIDTH, IMAGE_HEIGHT)

    matching_anchor_idxs, matching_gt_idxs = BoxUtils.find_anchor_gt_pairs(
        anchors, [letterbox_objects.boxes], 0.3
    )
    background_anchor_idxs = BoxUtils.find_anchors_meeting_iou_condition(
        anchors, [letterbox_objects.boxes], 0.05, False
    )
    anchor_idxs = matching_anchor_idxs[0]
    gt_idxs = matching_gt_idxs[0]

    unique_gts = set(gt_idxs.unique().cpu().tolist())
    expected_gts = set(range(objects.boxes.shape[0]))
    missed_gts = expected_gts.difference(unique_gts)
    anchors_per_gt = gt_idxs.bincount(minlength=objects.boxes.shape[0]).cpu().tolist()

    if all_boxes is None:
        all_boxes = letterbox_objects.boxes
    else:
        all_boxes = torch.cat((all_boxes, letterbox_objects.boxes), dim=0)

    # Log out when we have not associated a GT box with an anchor
    # if len(missed_gts) != 0:
    #     print(label_file)
    #     print("Num ground truths:", objects.boxes.shape[0])
    #     print("Matched ground truths:", len(unique_gts))
    #     print("Missed GT:", expected_gts.difference(unique_gts))
    #     print("Anchors per GT:", anchors_per_gt)

    num_anchors_per_gt += anchors_per_gt
    num_objects += objects.boxes.shape[0]
    missed_objects += len(missed_gts)

    classed_anchors += gt_idxs.shape[0]
    background_anchors += background_anchor_idxs[0].shape[0]
    empty_anchors += 8732 - (gt_idxs.shape[0] + background_anchor_idxs[0].shape[0])

In [ ]:
total_anchors = classed_anchors + background_anchors + empty_anchors
print("Classed anchors:", classed_anchors)
print("Background anchors:", background_anchors)
print("Empty anchors:", empty_anchors)
print("Total anchors:", total_anchors)
print("Anchors f:", total_anchors / (SAMPLE_SIZE * 8732))

print("Classed fraction:", classed_anchors / total_anchors)
print("Background fraction:", background_anchors / total_anchors)
print("Empty fraction:", empty_anchors / total_anchors)

### Investigate GT misses
It is possible for GT boxes to not get associated with any anchor box. This is because other GT boxes have already been associated with this box.

This occurs for two reasons:
1. The anchor boxes are not representative of the shape/size of the GT boxes - resulting in lots of poor matches to the same anchor boxes.
2. GT boxes having high IoU between themselves.

In [ ]:
print("Total number of objects:", num_objects)
print("Number of missed GT boxes:", missed_objects)

### Investigate number of anchors per GT
We want to know the distribution of the number of anchor boxes associated with each GT box.

By having vastly different anchors per GT we will unfairly biases some objects in the image.

In [ ]:
plt.figure()
plt.hist(num_anchors_per_gt, max(num_anchors_per_gt))
plt.grid()
plt.xlim((0, max(num_anchors_per_gt)))
plt.xlabel("Anchors per GT")
plt.ylabel("Occurrences")
plt.title("Distribution of anchors per GT box")

In [ ]:
anchors_per_gt = np.array(num_anchors_per_gt)

print(f"Total number of GT boxes: {num_objects}")
print(f"Median: {np.median(anchors_per_gt):.2f}")
print(f"Mean: {anchors_per_gt.mean():.2f}")
print(f"STD: {anchors_per_gt.std():.2f}")
print(f"Minimum: {anchors_per_gt.min()}")
print(f"Maximum: {anchors_per_gt.max()}")

### Investigate GT shape

In [ ]:
cxs = all_boxes[:, 0].cpu().numpy()

plt.figure()
plt.hist(cxs, bins=60)
plt.grid()
plt.xlabel("cx")
plt.xlim((0, 1))
plt.ylabel("Occurrences")
plt.title("Distribution of cx positions")

In [ ]:
cys = all_boxes[:, 1].cpu().numpy()

plt.figure()
plt.hist(cys, bins=60)
plt.grid()
plt.xlabel("cy")
plt.xlim((0, 1))
plt.ylabel("Occurrences")
plt.title("Distribution of cy positions")

In [ ]:
widths = all_boxes[:, 2].cpu().numpy()
anchor_widths = anchors[0, :, 2].cpu().numpy()

plt.figure()
plt.hist(widths, bins=60, label="Object widths")
plt.hist(anchor_widths, bins=60, label="Anchor widths")
plt.grid()
plt.xlabel("Width")
plt.xlim(left=0)
plt.ylabel("Occurrences")
plt.title("Distribution of widths")
plt.legend()

In [ ]:
heights = all_boxes[:, 3].cpu().numpy()
anchor_heights = anchors[0, :, 3].cpu().numpy()

plt.figure()
plt.hist(heights, bins=60, label="Object heights")
plt.hist(anchor_heights, bins=60, label="Anchor heights")
plt.grid()
plt.xlabel("Height")
plt.xlim(left=0)
plt.ylabel("Occurrences")
plt.title("Distribution of heights")
plt.legend()